In [ ]:
%pylab
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(1,'../src')
sys.path.insert(1,'../')

In [ ]:
# matplotlib.rcParams['savefig.dpi'] = 1.5 * matplotlib.rcParams['savefig.dpi']
from __future__ import print_function
from __future__ import division
import timeit
from ellipsoid import *
from newfuns import *

In [ ]:
# We first consider the distributions taken in ADLS namely

mixture_params = ((.65, -.45, .15), (.35, .3, .2))
gmm = mixture_distribution([normal_distribution(-.45, .15), normal_distribution(.3, .2)], [.65, .35])
fig = plot_distribution(gmm, (-1, 1))

beta = mixture_distribution([beta_distribution(.8, 4), beta_distribution(2, 2)], [.4, .6])
fig = plot_distribution(beta, (0, 1), color='green')

gamma = mixture_distribution([gamma_distribution(2.0, 2.0), gamma_distribution(7.5, 1.0)], [.7, .3])
fig = plot_distribution(gamma, (0, 20), color='red')

In [ ]:
def run_experiment(distribution, estimator, n_vals, num_trials):
    ret_time = np.zeros((len(n_vals), num_trials))
    ret_error = np.zeros((len(n_vals), num_trials))
    overall_start = timeit.default_timer()
    for ii, n in enumerate(n_vals):
        print('n = {} ... '.format(n), end='')
        start = timeit.default_timer()
        total_estimation_time = 0.0
        for jj in range(num_trials):
            samples = sorted(distribution.draw_samples(n))
            result, time = estimator(samples)
            total_estimation_time += time
            ret_time[ii, jj] = time
            ret_error[ii, jj] = piecewise_poly_error_function(result, samples, distribution)
        end = timeit.default_timer()
        print('total time: {} seconds, total estimation time: {} seconds.'.format(float(end - start), total_estimation_time))
    overall_stop = timeit.default_timer()
    print('Experiment took {} seconds'.format(float(overall_stop - overall_start)))
    return ret_time, ret_error


def pp_poly_estimator(distribution, samples, t, d, num_initial_pieces):
    n = len(samples)
    smin = np.min(samples) - 1e-3
    smax = np.max(samples) + 1e-3
    U = d * d * (math.sqrt(2) + 1.0)**d
    eps = math.sqrt(t * d / float(n))
#     print('eps estimate = {}'.format(eps))
    akproj_gap_tolerance = eps * eps * 0.5
#     print('akproj_gap_tolerance = {}'.format(akproj_gap_tolerance))
    #num_pieces = int(math.ceil(t / eps))s
#     print('Upper bound = {}  Num initial pieces = {}'.format(U, num_initial_pieces))
#     cvxopt.solvers.options['show_progress'] = False

    #Vai: what is this tolerance U? seems to be set to d^2(2.4142)^d
    start = timeit.default_timer()
    res = pp_learning(t, d, num_initial_pieces, (smin, smax), samples, verbose=0, akproj_num_iter=200, akproj_upper_bound=U, akproj_gap_tolerance=akproj_gap_tolerance)
    new_res = NewHypothesisPieces(res, t, d, samples)
    stop = timeit.default_timer()
    err_old = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(res), (smin, smax))
    err_new = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(new_res), (smin, smax))
    return [err_old, err_new], float(stop - start)

    
def piecewise_linear_estimator(samples):
    n = len(samples)
    smin = np.min(samples) - 1e-3
    smax = np.max(samples) + 1e-3
    samples_cpp = ellipsoid_cpp.DoubleVector(samples)
    
    t = 20
    eps = math.sqrt(2 * t / float(n))
    num_initial_intervals = min(n, int(math.ceil(t / eps)))
    num_merged_intervals_holdout = t
    max_final_num_intervals = 2 * num_merged_intervals_holdout + 1

    h = ellipsoid_cpp.LinearPzieceVector()
    opts = ellipsoid_cpp.A1ProjectionOptions()
    opts.max_gap = eps / (4.0 * max_final_num_intervals)
    opts.max_num_iterations = 20
    opts.num_initial_interval_levels = 2
    stats = ellipsoid_cpp.A1ProjectionStats()

    start = timeit.default_timer()
    ellipsoid_cpp.piecewise_linear_approx(samples_cpp, smin, smax, num_initial_intervals, num_merged_intervals_holdout, max_final_num_intervals, opts, h, stats)
    stop = timeit.default_timer()
    
    return convert_piecewise_linear_to_pp_hypothesis(h), float(stop - start)

def pp_poly_estimate_explicit(distribution, samples, t, d, num_initial_pieces):
    n = len(samples)
    smin = np.min(samples) - 1e-3
    smax = np.max(samples) + 1e-3
    U = d * d * (math.sqrt(2) + 1.0)**d
    eps = math.sqrt(t * d / float(n))
#     print('eps estimate = {}'.format(eps))
    akproj_gap_tolerance = eps * eps * 0.5
#     print('akproj_gap_tolerance = {}'.format(akproj_gap_tolerance))
    #num_pieces = int(math.ceil(t / eps))s
#     print('Upper bound = {}  Num initial pieces = {}'.format(U, num_initial_pieces))
#     cvxopt.solvers.options['show_progress'] = False

    #Vai: what is this tolerance U? seems to be set to d^2(2.4142)^d
    start = timeit.default_timer()
#     print('ululu', t, d, num_initial_pieces)
    res = pp_learning(t, d, num_initial_pieces, (smin, smax), samples, verbose=0, akproj_num_iter=200, akproj_upper_bound=U, akproj_gap_tolerance=akproj_gap_tolerance)
#     print('heeehaa')
    new_res = NewHypothesisPieces(res, t, d, samples)
    stop = timeit.default_timer()
#     err_old = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(res), (smin, smax))
#     err_new = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(new_res), (smin, smax))
    return [res, new_res], float(stop - start)


# distribution takes distribution input, set estimator = pp_poly_estimator
# n_vals is the list of samples to use e.g. [1000, 2000]
# num_trials is number of trivals per sample, e.g. 10 or 20
# t and d are the usual (function below is not crossvalidated)
# num_initial_pieces is the number of initial pieces, take it to be min(n,20*t)

def piecewise_poly_error_function(h, samples, distribution):
    smin = np.min(samples) - 1e-3
    smax = np.max(samples) + 1e-3
    return compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(h), (smin, smax))

def new_experiment(distribution, estimator, n_vals, num_trials, t, d, num_initial_pieces):
    ret_times = []
    ret_error_old = []
    ret_error_new = []
    for ii, n in enumerate(n_vals):
        print('Evaluating the {} distribution with {} samples by averaging over {} runs'.format(distribution[0], n, num_trials))
        start = timeit.default_timer()
        ret_error_old += [0]
        ret_error_new += [0]
        estimation_time = 0
        for jj in range(num_trials):
            samples = sorted(distribution[1].draw_samples(n))
            if estimator == pp_poly_estimator:
                resses, time = estimator(distribution[1], samples, t, d, num_initial_pieces)
            estimation_time += time
            ret_error_old[ii] += resses[0]/num_trials
            ret_error_new[ii] += resses[1]/num_trials
        ret_times += [estimation_time]
        print('Error incurred by ADLS is {} and TURF is {}'.format(ret_error_old[ii], ret_error_new[ii]))
        print('Time over {} runs is {} seconds \n'.format(num_trials, estimation_time))
    print('Total experiment took {} seconds \n'.format(sum(ret_times)))
    return zip(n_vals, ret_error_old, ret_error_new, ret_times)

def newer_cv_experiment(distribution, n_vals, num_trials, d, param):
    ret_times = []
    ret_error_old = []
    ret_error_new = []
    for ii, n in enumerate(n_vals):
        print('Evaluating the {} distribution with {} samples by averaging over {} runs'.format(distribution[0], n, num_trials))
        start = timeit.default_timer()
        ret_error_old += [0]
        ret_error_new += [0]
        estimation_time = 0
        # set number of initial pieces to be 100, matching ADLS
        num_initial_pieces = 100
#         print("Number of initial pieces is", num_initial_pieces)
#         print('huhu', num_initial_pieces)
        # tceil is set at sqrt since roughly speaking with more than sqrt{n} intervals even collisions won't happen
        tceil_exp = int(np.log2(min(np.sqrt(n),100)))
        for jj in range(num_trials):
#             print('Running trial {} with {} samples, number of initial pieces {}'.format(jj, n, num_initial_pieces))
            samples = sorted(distribution[1].draw_samples(n))
            resses, time = toss_validator(distribution[1], samples, tceil_exp, d, param, num_initial_pieces)
            estimation_time += time
            ret_error_old[ii] += resses[0]/num_trials
            ret_error_new[ii] += resses[1]/num_trials
        ret_times += [estimation_time]
        print('Error incurred by ADLS is {} and TURF is {}'.format(ret_error_old[ii], ret_error_new[ii]))
        print('Time over {} runs is {} seconds \n'.format(num_trials, estimation_time))
    print('Total experiment took {} seconds \n'.format(sum(ret_times)))
    return zip(n_vals, ret_error_old, ret_error_new, ret_times)


def toss_validator(distribution, samples, tceil_exp, d, param, num_initial_pieces):
    smin = np.min(samples) - 1e-3
    smax = np.max(samples) + 1e-3
    tot_time = 0
    time_est = 0
    time_cv = 0
    best_err_old = 2
    best_err_new = 2
    const = param
    estim_set = []
    for t in range(tceil_exp):
#         num_initial_pieces = 50
#         print('lalallalala', 2**(t+1), num_initial_pieces)
# number of pieces varies between 2 and 2^tceil_exp
        estes, time = pp_poly_estimate_explicit(distribution, samples, 2**(t+1), d, num_initial_pieces)
        estim_set = estim_set + [estes]
        time_est += time
#     print('Time taken to compute all estimators is ', time_est)

    start = timeit.default_timer()
    
    flag_old = 0
    for t in range(tceil_exp):
        cur_estim_old = estim_set[t][0]
        if flag_old == 1:
            break
#         print('Current ADLS candidate with d = {0} and t = {1}'.format(d, 2**(t+1)))
        for tt in range(t, tceil_exp):
#             print('bobo')
            check_estim_old = estim_set[tt][0]
            delta_old = compute_l1_quad(get_ppoly_pdf(cur_estim_old), get_ppoly_pdf(check_estim_old), (smin, smax))
# CV parameter is prop to sqrt of the number of pieces used - a meaningful heauristic
            thresh_old = param*np.sqrt(len(check_estim_old))*np.sqrt((2**d+1)/len(samples))
            if delta_old > thresh_old:
                break
# flag takes value 1 if checks have passed for all tt>t
            if tt == tceil_exp - 1:
                flag_old = 1
#                 print('Value of t that passed all tests is', t)
                

    flag_new = 0          
    for t in range(tceil_exp):
        cur_estim_new = estim_set[t][1]
        if flag_new == 1:
            break
#         print('Current TURF candidate with d = {0} and t = {1}'.format(d, 2**(t+1)))
        for tt in range(t, tceil_exp):
#             print('bobobo')
            check_estim_new = estim_set[tt][1]
            delta_new = compute_l1_quad(get_ppoly_pdf(cur_estim_new), get_ppoly_pdf(check_estim_new), (smin, smax))
# CV parameter is prop to sqrt of the number of pieces used - a meaningful heauristic
            thresh_new = param*np.sqrt(len(check_estim_new))*np.sqrt((2**d+1)/len(samples))
            if delta_new > thresh_new:
                break
# flag takes value 1 if checks have passed for all tt>t
            if tt == tceil_exp - 1:
                flag_new = 1
#                 print('Value of t that passed all tests is', t)
    stop = timeit.default_timer()
    time_cv = stop - start
#     print('Time taken to cross validate is', time_cv)
    tot_time = time_est + time_cv
    best_err_old = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(cur_estim_old), (smin, smax))
    best_err_new = compute_l1_quad(distribution.get_pdf(), get_ppoly_pdf(cur_estim_new), (smin, smax))  
    return [best_err_old, best_err_new], tot_time

We now consider noisy distributions, i.e. same as above but perturbed by Gaussian noise

In [ ]:
# generating noisy distributions
# k refers to the number of Gaussian noise components
k = 100

means = list(np.random.uniform(-1,1,k))
mix = []
dist_mix = []
for i in range(k):
    mix += [0.25/k]
    dist_mix += [normal_distribution(means[i], 0.1/np.sqrt(k))]

# noise is added to the ADLS distributions, defined again for completeness

mixture_params = ((.65, -.45, .15), (.35, .3, .2))
gmm = mixture_distribution([normal_distribution(-.45, .15), normal_distribution(.3, .2)], [.65, .35])

full_mix = [0.75] + mix
full_dist_mix = [gmm] + dist_mix
gmmk = mixture_distribution(full_dist_mix, full_mix)
fig = plot_distribution(gmmk, (-1,1))


means = list(np.random.uniform(0,1,k))
mix = []
dist_mix = []
for i in range(k):
    mix += [0.25/k]
    dist_mix += [normal_distribution(means[i],.05/np.sqrt(k))]

beta = mixture_distribution([beta_distribution(.8, 4), beta_distribution(2, 2)], [.4, .6])

full_mix = [0.75] + mix
full_dist_mix = [beta] + dist_mix
betak = mixture_distribution(full_dist_mix, full_mix)
fig = plot_distribution(betak, (0,1), color='green')


means = list(np.random.uniform(0,20,k))
mix = []
dist_mix = []
for i in range(k):
    mix += [0.25/k]
    dist_mix += [normal_distribution(means[i], 1/np.sqrt(k))]
    
gamma = mixture_distribution([gamma_distribution(2.0, 2.0), gamma_distribution(7.5, 1.0)], [.7, .3])

full_mix = [0.75] + mix
full_dist_mix = [gamma] + dist_mix
gammak = mixture_distribution(full_dist_mix, full_mix)
fig = plot_distribution(gammak, (0,20), color='red')

In [ ]:
# cv over t
# n_vals = [1000, 2000]
n_vals = [1000, 2000, 5000, 10000, 20000, 40000, 80000]
num_trials = 100
distributions = [('beta', beta), ('gamma', gamma), ('gmm', gmm)]

# greatest t, tceil
# fixed d
d = 2
# param is the multiple used in CV. A large value leads to few pieces
param = 5
for dist in distributions:
    expt_out = newer_cv_experiment(dist, n_vals, num_trials, d, param)
    res_adls = []
    res_turf = []
    for vec in expt_out:
        res_adls += [(vec[0], vec[1])]
        res_turf += [(vec[0], vec[2])]
    print('ADLS with d={0} on {1}'.format(d, dist[0]))
    print(res_adls)
    print('TURF with d={0} on {1}'.format(d, dist[0]))
    print(res_turf)
    print('\n')
    print('-----------------------------------------------------------------')
    print('\n')

In [ ]:
# cv over t
# n_vals = [1000, 2000]
n_vals = [1000, 2000, 5000, 10000, 20000, 40000, 80000]
num_trials = 100
distributions = [('betak', betak), ('gammak', gammak), ('gmm', gmmk)]

# greatest t, tceil
# fixed d
d = 2
# param is the multiple used in CV. A large value leads to few pieces
param = 5
for dist in distributions:
    expt_out = newer_cv_experiment(dist, n_vals, num_trials, d, param)
    res_adls = []
    res_turf = []
    for vec in expt_out:
        res_adls += [(vec[0], vec[1])]
        res_turf += [(vec[0], vec[2])]
    print('ADLS with d={0} on {1}'.format(d, dist[0]))
    print(res_adls)
    print('TURF with d={0} on {1}'.format(d, dist[0]))
    print(res_turf)
    print('\n')
    print('-----------------------------------------------------------------')
    print('\n')